In [1]:
import linkpred
from linkpred.evaluation import Pair
import networkx as nx
import random
import itertools
from operator import itemgetter
import pickle
import matplotlib.pyplot as plt
from sklearn.metrics import auc

In [12]:
import networkx as nx
import pandas as pd
import numpy as np
import json
import cdlib
from cdlib import evaluation
from cdlib import algorithms
from cdlib import viz
from cdlib import ensemble
from cdlib import readwrite
import pickle
import pquality
import matplotlib.pyplot as plt
%matplotlib inline
from nf1 import NF1
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from operator import itemgetter
import powerlaw
from networkx.drawing.nx_agraph import graphviz_layout
import pickle
from statistics import mean, stdev
import seaborn as sns
import json
import random

In [3]:
path = "../DataSet StackOverflow/"

In [4]:
path_graph = "../DataSet StackOverflow/Graph_data/"

In [5]:
saved_path = "../DataSet StackOverflow/LinkPrediction_task/communities/"

In [6]:
G = nx.read_graphml(path_graph+"Real_Network.graphml")

In [7]:
nx.info(G)

'Name: \nType: Graph\nNumber of nodes: 13780\nNumber of edges: 97892\nAverage degree:  14.2078'

In [8]:
community_path = "../DataSet StackOverflow/Community_task/"

In [9]:
with open(community_path+'louvain.pickle', 'rb') as f:
    louvain_coms = pickle.load(f)

In [15]:
for i in louvain_coms.communities[:9]:
    print(len(i))

2219
1850
1528
1514
1234
1208
1144
914


In [16]:
def prepare_data(G, idx):
    training_edge_len = len(G.edges()) - int(len(G.edges())/100 * 20)
    test_edge_len = len(G.edges()) - training_edge_len
    
    training_edges = random.sample(G.edges, training_edge_len)
    
    training = G.edge_subgraph(training_edges)
    
    test = G.copy()
    test.remove_edges_from(training.edges())
    
    nodes = list(G.nodes())
    nodes.extend(list(test.nodes()))
    
    test = [Pair(i) for i in test.edges()]
    universe = set([Pair(i) for i in itertools.product(nodes, nodes) if i[0]!=i[1]])
    print(f" -- Storing for community {idx}")
    with open(saved_path+f'universe_{idx}.pickle', 'wb') as f:
        pickle.dump(universe, f)
    with open(saved_path+f'test_{idx}.pickle', 'wb') as f:
        pickle.dump(test, f) 
    nx.write_graphml(training, saved_path+f"training_{idx}.graphml")

In [18]:
for idx, com in enumerate(louvain_coms.communities[:9]):
    sampled_nodes = random.sample(G.subgraph(com).nodes, 500)
    prepare_data(G.subgraph(sampled_nodes), idx)

 -- Storing for community 0
 -- Storing for community 1
 -- Storing for community 2
 -- Storing for community 3
 -- Storing for community 4
 -- Storing for community 5
 -- Storing for community 6
 -- Storing for community 7
 -- Storing for community 8


In [21]:
def predict(idx):
    with open(saved_path+f'universe_{idx}.pickle', 'rb') as f:
        universe = pickle.load(f)
    with open(saved_path+f'test_{idx}.pickle', 'rb') as f:
        test = pickle.load(f)
    training = nx.read_graphml(saved_path+f"training_{idx}.graphml")
    
    print(f"-- For {idx} predict CommonNeighbours")
    
    cn = linkpred.predictors.CommonNeighbours(training, excluded=training.edges())
    cn_results = cn.predict()
    
    res = []
    for i, j in zip(cn_results, cn_results.values()):
        res.append([Pair(i), j])
   
    with open(saved_path+f'cn_results_{idx}.pickle', 'wb') as f:
        pickle.dump(res, f)
        
        
    print(f"-- For {idx} predict AdamicAdar")
    aa = linkpred.predictors.AdamicAdar(training, excluded=training.edges())
    aa_results = aa.predict()
    
    
    res = []
    for i, j in zip(aa_results, aa_results.values()):
        res.append([Pair(i), j])
   
    with open(saved_path+f'aa_results_{idx}.pickle', 'wb') as f:
        pickle.dump(res, f)
        
    print(f"-- For {idx} predict Jaccard")
    
    jc = linkpred.predictors.Jaccard(training, excluded=training.edges())
    jc_results = jc.predict()
    
    
    res = []
    for i, j in zip(jc_results, jc_results.values()):
        res.append([Pair(i), j])

    with open(saved_path+f'jc_results_{idx}.pickle', 'wb') as f:
        pickle.dump(res, f)
    
    print(f"-- For {idx} evaluate CommonNeighbour")
        
    cn_evaluation = linkpred.evaluation.EvaluationSheet(cn_results, test, universe)

    with open(saved_path+f'cn_evaluation_{idx}.pickle', 'wb') as f:
        pickle.dump(cn_evaluation, f) 

    print(f"-- For {idx} evaluate AdamicAdar")
        
    aa_evaluation = linkpred.evaluation.EvaluationSheet(aa_results, test, universe)

    with open(saved_path+f'aa_evaluation_{idx}.pickle', 'wb') as f:
        pickle.dump(aa_evaluation, f) 
        
        
    print(f"-- For {idx} evaluate Jaccard")
        
    jc_evaluation = linkpred.evaluation.EvaluationSheet(jc_results, test, universe)

    with open(saved_path+f'jc_evaluation_{idx}.pickle', 'wb') as f:
        pickle.dump(jc_evaluation, f) 

In [22]:
def evaluate(idx):
    
    with open(saved_path+f'cn_evaluation_{idx}.pickle', 'rb') as f:
        cn_evaluation = pickle.load(f)
    with open(saved_path+f'aa_evaluation_{idx}.pickle', 'rb') as f:
        aa_evaluation = pickle.load(f)
    with open(saved_path+f'jc_evaluation_{idx}.pickle', 'rb') as f:
        jc_evaluation = pickle.load(f)

    
    print("Area Under Roc Curve (AUROC)")
    print(f"CommonNeighbour: \t {auc(cn_evaluation.fallout(), cn_evaluation.recall())}")
    print(f"AdamicAdar: \t {auc(aa_evaluation.fallout(), aa_evaluation.recall())}")
    print(f"Jaccard: \t {auc(jc_evaluation.fallout(), jc_evaluation.recall())}")
    
    
    #plt.plot(jc_evaluation.fallout(), jc_evaluation.recall(), label="Jaccard")

    #plt.xlabel("FPR")
    #plt.ylabel("TPR")
    #plt.legend()
    #plt.show()
    

In [23]:
predict(0)

-- For 0 predict CommonNeighbours
-- For 0 predict AdamicAdar
-- For 0 predict Jaccard
-- For 0 evaluate CommonNeighbour
-- For 0 evaluate AdamicAdar
-- For 0 evaluate Jaccard


In [24]:
evaluate(0)

Area Under Roc Curve (AUROC)
CommonNeighbour: 	 0.014611802217855461
AdamicAdar: 	 0.0190591418326423
Jaccard: 	 0.004636067582722781


In [25]:
predict(1)

-- For 1 predict CommonNeighbours
-- For 1 predict AdamicAdar
-- For 1 predict Jaccard
-- For 1 evaluate CommonNeighbour
-- For 1 evaluate AdamicAdar
-- For 1 evaluate Jaccard


In [26]:
evaluate(1)

Area Under Roc Curve (AUROC)
CommonNeighbour: 	 0.043849792931871606
AdamicAdar: 	 0.05469709108368652
Jaccard: 	 0.014099672205548846


In [27]:
predict(2)

-- For 2 predict CommonNeighbours
-- For 2 predict AdamicAdar
-- For 2 predict Jaccard
-- For 2 evaluate CommonNeighbour
-- For 2 evaluate AdamicAdar
-- For 2 evaluate Jaccard


In [28]:
evaluate(2)

Area Under Roc Curve (AUROC)
CommonNeighbour: 	 0.15245817369768105
AdamicAdar: 	 0.18336498910936233
Jaccard: 	 0.039878716837338776


In [29]:
predict(3)

-- For 3 predict CommonNeighbours
-- For 3 predict AdamicAdar
-- For 3 predict Jaccard
-- For 3 evaluate CommonNeighbour
-- For 3 evaluate AdamicAdar
-- For 3 evaluate Jaccard


In [30]:
evaluate(3)

Area Under Roc Curve (AUROC)
CommonNeighbour: 	 0.04700709207758634
AdamicAdar: 	 0.055791863362182004
Jaccard: 	 0.02199092658306438


In [31]:
predict(4)

-- For 4 predict CommonNeighbours
-- For 4 predict AdamicAdar
-- For 4 predict Jaccard
-- For 4 evaluate CommonNeighbour
-- For 4 evaluate AdamicAdar
-- For 4 evaluate Jaccard


In [32]:
evaluate(4)

Area Under Roc Curve (AUROC)
CommonNeighbour: 	 0.017317231695085256
AdamicAdar: 	 0.020819193580742225
Jaccard: 	 0.009571819458375125


In [33]:
predict(5)

-- For 5 predict CommonNeighbours
-- For 5 predict AdamicAdar
-- For 5 predict Jaccard
-- For 5 evaluate CommonNeighbour
-- For 5 evaluate AdamicAdar
-- For 5 evaluate Jaccard


In [34]:
evaluate(5)

Area Under Roc Curve (AUROC)
CommonNeighbour: 	 0.019945650881809624
AdamicAdar: 	 0.02444201397790601
Jaccard: 	 0.008297592152552038


In [35]:
predict(6)

-- For 6 predict CommonNeighbours
-- For 6 predict AdamicAdar
-- For 6 predict Jaccard
-- For 6 evaluate CommonNeighbour
-- For 6 evaluate AdamicAdar
-- For 6 evaluate Jaccard


In [36]:
evaluate(6)

Area Under Roc Curve (AUROC)
CommonNeighbour: 	 0.024456227112956804
AdamicAdar: 	 0.02890317587222588
Jaccard: 	 0.0090478730589926


In [37]:
predict(7)

-- For 7 predict CommonNeighbours
-- For 7 predict AdamicAdar
-- For 7 predict Jaccard
-- For 7 evaluate CommonNeighbour
-- For 7 evaluate AdamicAdar
-- For 7 evaluate Jaccard


In [38]:
evaluate(7)

Area Under Roc Curve (AUROC)
CommonNeighbour: 	 0.0056600997739704945
AdamicAdar: 	 0.008273898421954433
Jaccard: 	 0.005569628529946308


In [39]:
predict(8)

-- For 8 predict CommonNeighbours
-- For 8 predict AdamicAdar
-- For 8 predict Jaccard
-- For 8 evaluate CommonNeighbour
-- For 8 evaluate AdamicAdar
-- For 8 evaluate Jaccard


In [40]:
evaluate(8)

Area Under Roc Curve (AUROC)
CommonNeighbour: 	 0.06631589840209533
AdamicAdar: 	 0.08553909737752631
Jaccard: 	 0.030996789463132845


In [41]:
def predict2(idx):
    with open(saved_path+f'universe_{idx}.pickle', 'rb') as f:
        universe = pickle.load(f)
    with open(saved_path+f'test_{idx}.pickle', 'rb') as f:
        test = pickle.load(f)
    training = nx.read_graphml(saved_path+f"training_{idx}.graphml")
    
    print(f"-- For {idx} predict Katz")
    
    kz = linkpred.predictors.Katz(training, excluded=training.edges())
    kz_results = kz.predict()
    
    res = []
    for i, j in zip(kz_results, kz_results.values()):
        res.append([Pair(i), j])
   
    with open(saved_path+f'kz_results_{idx}.pickle', 'wb') as f:
        pickle.dump(res, f)
        
        
    print(f"-- For {idx} predict SimRank")
    simrank = linkpred.predictors.SimRank(training, excluded=training.edges())
    simrank_results = simrank.predict()
    
    
    res = []
    for i, j in zip(simrank_results, simrank_results.values()):
        res.append([Pair(i), j])
   
    with open(saved_path+f'simrank_results_{idx}.pickle', 'wb') as f:
        pickle.dump(res, f)
        

    print(f"-- For {idx} evaluate Katz")
        
    kz_evaluation = linkpred.evaluation.EvaluationSheet(kz_results, test, universe)

    with open(saved_path+f'kz_evaluation_{idx}.pickle', 'wb') as f:
        pickle.dump(kz_evaluation, f) 

    print(f"-- For {idx} evaluate SimRank")
        
    simrank_evaluation = linkpred.evaluation.EvaluationSheet(simrank_results, test, universe)

    with open(saved_path+f'simrank_evaluation_{idx}.pickle', 'wb') as f:
        pickle.dump(simrank_evaluation, f) 

In [42]:
def evaluate2(idx):
    
    with open(saved_path+f'kz_evaluation_{idx}.pickle', 'rb') as f:
        kz_evaluation = pickle.load(f)
    with open(saved_path+f'simrank_evaluation_{idx}.pickle', 'rb') as f:
        sm_evaluation = pickle.load(f)

    
    print("Area Under Roc Curve (AUROC)")
    print(f"Katz: \t {auc(kz_evaluation.fallout(), kz_evaluation.recall())}")
    print(f"SimRank: \t {auc(sm_evaluation.fallout(), sm_evaluation.recall())}")

In [43]:
predict2(0)

-- For 0 predict Katz
Computing matrix powers: [############################################################] 5/5
-- For 0 predict SimRank
-- For 0 evaluate Katz
-- For 0 evaluate SimRank


In [44]:
evaluate2(0)

Area Under Roc Curve (AUROC)
Katz: 	 0.125922302697261
SimRank: 	 0.10792348478885144


In [45]:
predict2(1)
evaluate2(1)

-- For 1 predict Katz
Computing matrix powers: [############################################################] 5/5
-- For 1 predict SimRank
-- For 1 evaluate Katz
-- For 1 evaluate SimRank
Area Under Roc Curve (AUROC)
Katz: 	 0.24146000672469833
SimRank: 	 0.18306726497110176


In [46]:
predict2(2)
evaluate2(2)

-- For 2 predict Katz
Computing matrix powers: [############################################################] 5/5
-- For 2 predict SimRank
-- For 2 evaluate Katz
-- For 2 evaluate SimRank
Area Under Roc Curve (AUROC)
Katz: 	 0.4702632682316822
SimRank: 	 0.3142765124783215


In [47]:
predict2(3)
evaluate2(3)

-- For 3 predict Katz
Computing matrix powers: [############################################################] 5/5
-- For 3 predict SimRank
-- For 3 evaluate Katz
-- For 3 evaluate SimRank
Area Under Roc Curve (AUROC)
Katz: 	 0.3933913233861601
SimRank: 	 0.35465447488987734


In [48]:
predict2(4)
evaluate2(4)

-- For 4 predict Katz
Computing matrix powers: [############################################################] 5/5
-- For 4 predict SimRank
-- For 4 evaluate Katz
-- For 4 evaluate SimRank
Area Under Roc Curve (AUROC)
Katz: 	 0.15970741023069207
SimRank: 	 0.14666854964894682


In [49]:
predict2(5)
evaluate2(5)

-- For 5 predict Katz
Computing matrix powers: [############################################################] 5/5
-- For 5 predict SimRank
-- For 5 evaluate Katz
-- For 5 evaluate SimRank
Area Under Roc Curve (AUROC)
Katz: 	 0.16639980850691258
SimRank: 	 0.15571517729193474


In [50]:
predict2(6)
evaluate2(6)

-- For 6 predict Katz
Computing matrix powers: [############################################################] 5/5
-- For 6 predict SimRank
-- For 6 evaluate Katz
-- For 6 evaluate SimRank
Area Under Roc Curve (AUROC)
Katz: 	 0.17788766434490616
SimRank: 	 0.15835087128012862


In [51]:
predict2(7)
evaluate2(7)

-- For 7 predict Katz
Computing matrix powers: [############################################################] 5/5
-- For 7 predict SimRank
-- For 7 evaluate Katz
-- For 7 evaluate SimRank
Area Under Roc Curve (AUROC)
Katz: 	 0.17686646723761035
SimRank: 	 0.23126114358149025


In [52]:
predict2(8)
evaluate2(8)

-- For 8 predict Katz
Computing matrix powers: [############################################################] 5/5
-- For 8 predict SimRank
-- For 8 evaluate Katz
-- For 8 evaluate SimRank
Area Under Roc Curve (AUROC)
Katz: 	 0.44043366577328297
SimRank: 	 0.4252993030050002
